In [25]:
from __future__ import print_function

import pandas as pd
import numpy as np
import gensim
import string

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import get_file


In [8]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format(('fiction.lowercased.tokenized.word2vec.300d'),binary=False)

In [52]:
print(len(model.vocab))
print(len(model['i']))
model['i'][:10]

110363
300


array([-0.267473,  0.770113, -2.217801, -2.82495 ,  0.195229,  0.080929,
        0.576751,  0.906972,  1.862315, -0.97951 ], dtype=float32)

In [47]:
import numpy as np
arr=np.fromfile('uk_model.dat')
print(len(arr))
arr

43079169


array([ 2.16205267e-027,  2.03534858e+180,  9.03296881e+271, ...,
       -1.65833817e-303, -6.52529546e-295,  7.29293688e-304])

In [50]:
len(np.fromfile('fiction.lowercased.tokenized.word2vec.300d'))/300

131827.85333333333

In [28]:
url = 'https://raw.githubusercontent.com/maxim5/stanford-tensorflow-tutorials/master/data/arxiv_abstracts.txt'
path = get_file('arxiv_abstracts.txt', origin=url)

max_sentence_len = 40
with open(path) as file_:
    docs = file_.readlines()

In [130]:
with open('fiction.tokenized.shuffled.txt') as f:
    docs = f.readlines()
l=list()
for i in docs:
    if i!='\n' and i!='* * *\n':
        l.append(i)
max_sentence_len = 10
docs=l

In [16]:
# # with open('uk_model.dat') as f:
# #     d = f.readlines()
# d=np.fromfile('uk_model.dat')

In [131]:
sentences = [[word for word in doc.lower().translate(string.punctuation).split()[:max_sentence_len]] for doc in docs]
print('Num sentences:', len(sentences))

Num sentences: 1811548


In [ ]:
word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)
word_model.wv.vocab.keys() #watch words in our model

In [120]:
pretrained_weights = word_model.wv.vectors
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
print('Checking similar words:')
# for word in ['model', 'network', 'train', 'learn']:
for word in ['пришло', 'латинист', 'штука', 'глядела']:
    most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.wv.most_similar(word)[:8])
    print('  %s -> %s' % (word, most_similar))

Result embedding shape: (689, 20)
Checking similar words:
  пришло -> потому (1.00), штука (1.00), чтобы (1.00), мы (1.00), девица (1.00), повернулся (1.00), он (1.00), решил (1.00)
  латинист -> пытку. (1.00), раз (1.00), твоего (1.00), с (1.00), бы (1.00), воскликнул (1.00), любого (1.00), увернулся (1.00)
  штука -> девица (1.00), от (1.00), я, (1.00), получится (1.00), браннер (1.00), казалось, (1.00), взглядом (1.00), — (1.00)
  глядела -> пошел (0.88), собачился (0.88), назад, (0.88), толкнул!+ (0.88), понимал, (0.88), тряслись, (0.88), дураки, (0.88), перешептывались.+ (0.88)


In [75]:
def word2idx(word):
    return word_model.wv.vocab[word].index #get word by index:)
def idx2word(idx):
    return word_model.wv.index2word[idx] #get index by word:)

In [121]:
print('\nPreparing the data for LSTM...')
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence[:-1]):
        train_x[i, t] = word2idx(word)
        train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)


Preparing the data for LSTM...
train_x shape: (146, 10)
train_y shape: (146,)


In [122]:
print(sentences[0])
for t, word in enumerate(sentences[0][:-1]):
    print(word)

['послушай,', 'я', 'не', 'хотел', 'быть', 'полукровкой.+']
послушай,
я
не
хотел
быть


In [127]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [84]:
for i,j in enumerate(train_y):
    if i==10:
        break
    print(idx2word(j))

to
for
restarting
almost
rate
batch-processing
as
and
achievable
coordinates


In [129]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_next(text, num_generated=10):
    word_idxs = [word2idx(word) for word in text.lower().split()]
    for i in range(num_generated):
        prediction = model.predict(x=np.array(word_idxs))
        idx = sample(prediction[-1], temperature=0.7)
        word_idxs.append(idx)
    return ' '.join(idx2word(idx) for idx in word_idxs)

def on_epoch_end(epoch, _):
    print('\nGenerating text after epoch: %d' % epoch)
    texts = [
#     'deep convolutional',
        
        
#     'simple and effective',
#     'a nonconvex',
#     'a'
    'она'
    ]
    break_s='\n'+'-'*50+'\n'
    for text in texts:
        sample = generate_next(text)
        print('%s... -> %s%s' % (text, sample,break_s))

model.fit(train_x, train_y,
          batch_size=128,
          epochs=20,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Train on 146 samples
Epoch 1/20
128/146 [=========================>....] - ETA: 0s - loss: 6.5098
Generating text after epoch: 0
она... -> она раз не-а, убийственно, заткнешься?+ тебя остановился упс! воскликнул понял, умри,
--------------------------------------------------

146/146 [==============================] - 1s 4ms/sample - loss: 6.5073
Epoch 2/20
128/146 [=========================>....] - ETA: 0s - loss: 6.4952
Generating text after epoch: 1
она... -> она отозвался все, расселся мне пронзительно экскурсиях спокойствие. хоть снаружи швырялись
--------------------------------------------------

146/146 [==============================] - 1s 5ms/sample - loss: 6.4930
Epoch 3/20
128/146 [=========================>....] - ETA: 0s - loss: 6.4761
Generating text after epoch: 2
она... -> она доддз повторял авеню, просто вопрос, понятно. отозвался большинство самодовольно тем
--------------------------------------------------

146/146 [==============================] - 1s 4ms/sample -